In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

In [2]:
!unzip /content/drive/MyDrive/dacon/open.zip

Archive:  /content/drive/MyDrive/dacon/open.zip
   creating: open/
  inflating: open/train.csv          
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           


In [131]:
df_train = pd.read_csv('open/train.csv')
df_test = pd.read_csv('open/test.csv')
sub = pd.read_csv('open/sample_submission.csv')

# df_train = df_train.sort_values('begin_month', ascending=True).reset_index(drop=True)
# df_test = df_test.sort_values('begin_month', ascending=True).reset_index(drop=True)

df_train.loc[df_train['child_num'] > df_train['family_size'], 'family_size'] = 4
df_train.loc[(df_train['family_type']=='Married') & (df_train['family_size']<2), 'family_size'] = 3
df_train.loc[df_train['child_num'] == df_train['family_size'], 'family_size'] = 2

df_test.loc[df_test['child_num'] > df_test['family_size'], 'family_size'] = 4
df_test.loc[(df_test['family_type']=='Married') & (df_test['family_size']<2), 'family_size'] = 3

# temp = []
# duplicates = []
# for idx in tqdm(range(len(df_train))):
    # row = '_'.join(df_train.loc[idx, 'gender':'family_size'].astype(str).tolist())
#     if row in temp:
#         duplicates.append(sum([row==i for i in temp]) + 1)
#     else:
#         duplicates.append(1)
    # temp.append(row)

# df_train['rank'] = duplicates
# df_train['uid_rows'] = temp

# duplicates = []
# for idx in tqdm(range(len(df_test))):
#     row = '_'.join(df_test.loc[idx, 'gender':'family_size'].astype(str).tolist())
#     if row in temp:
#         duplicates.append(sum([row==i for i in temp]) + 1)
#     else:
#         duplicates.append(1)
#     temp.append(row)

# df_test['rank'] = duplicates
# df_test['uid_rows'] = temp[-10000:]
df_train['uid_rows'] = ['_'.join(df_train.loc[idx, 'gender':'family_size'].astype(str).tolist()) for idx in tqdm(range(len(df_train)))]
df_test['uid_rows'] = ['_'.join(df_test.loc[idx, 'gender':'family_size'].astype(str).tolist()) for idx in tqdm(range(len(df_test)))]

df_train = pd.concat([df_train, df_train.groupby('uid_rows')[['index']].rank().rename(columns={'index':'rank'})], 1)
df_test['rank'] = 0

train = df_train.copy()
test = df_test.copy()

# 부양가족
train['n_dependents2'] = train['family_size'] - train['child_num']
test['n_dependents2'] = test['family_size'] - test['child_num']

TARGET = 'credit'

for col in ['gender', 'car', 'reality', 'child_num', 'edu_type', 'house_type', 'occyp_type', 'income_type', 'family_type', 'work_phone', 'phone', 'email', 'family_size']:
    temp = train[col].value_counts(True).to_dict()
    for df in [train, test]:
        df[col] = df[col].map(temp)

test['uid_rows_vc'] = test['uid_rows'].map(train['uid_rows'].value_counts())
train['uid_rows_vc'] = train['uid_rows'].map(train['uid_rows'].value_counts())

train['DAYS_EMPLOYED_binary'] = train['DAYS_EMPLOYED']>0
test['DAYS_EMPLOYED_binary'] = test['DAYS_EMPLOYED']>0

train['DAYS_BIRTH_DATE'] = train['DAYS_BIRTH'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=25153, freq='d')[::-1])})
test['DAYS_BIRTH_DATE'] = test['DAYS_BIRTH'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=25153, freq='d')[::-1])})

# 미고용 상태 NULL
train['DAYS_EMPLOYED_DATE'] = train['DAYS_EMPLOYED'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=15714, freq='d')[::-1])})
test['DAYS_EMPLOYED_DATE'] = test['DAYS_EMPLOYED'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=15714, freq='d')[::-1])})
train['DAYS_EMPLOYED_DATE'] = pd.to_datetime(train['DAYS_EMPLOYED_DATE'].fillna('2020-01-01'))
test['DAYS_EMPLOYED_DATE'] = pd.to_datetime(test['DAYS_EMPLOYED_DATE'].fillna('2020-01-01'))


train['begin_month_DATE'] = train['begin_month'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=61, freq='m')[::-1])})
test['begin_month_DATE'] = test['begin_month'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=61, freq='m')[::-1])})

for df in [train, test]:

    # 2019년 기준 현재 나이, 정년까지 남은 연수
    df['age'] = 2019 - df['DAYS_BIRTH_DATE'].dt.year
    df['age2'] = 60 - df['age']
    df['income_total_y'] = df['income_total'] * (df['DAYS_EMPLOYED_DATE'].dt.year + df['DAYS_EMPLOYED_DATE'].dt.month)

    df['DAYS_BIRTH_month'] = df['DAYS_BIRTH_DATE'].dt.month
    df['DAYS_BIRTH_week'] = df['DAYS_BIRTH_DATE'].dt.isocalendar().week.astype(int)
    df['DAYS_BIRTH_day'] = df['DAYS_BIRTH_DATE'].dt.day
    df['DAYS_BIRTH_weekday'] = df['DAYS_BIRTH_DATE'].dt.weekday

    df['DAYS_EMPLOYED_DATE_month'] = df['DAYS_EMPLOYED_DATE'].dt.month
    df['DAYS_EMPLOYED_DATE_week'] = df['DAYS_EMPLOYED_DATE'].dt.isocalendar().week.astype(int)
    df['DAYS_EMPLOYED_DATE_day'] = df['DAYS_EMPLOYED_DATE'].dt.day
    df['DAYS_EMPLOYED_DATE_weekday'] = df['DAYS_EMPLOYED_DATE'].dt.weekday

    df['employment_days'] = (df['DAYS_BIRTH_DATE'] - df['DAYS_EMPLOYED_DATE']).dt.days
    df['employment_days'] = df['employment_days']//12

    df.loc[df['DAYS_EMPLOYED_DATE']=='2020-01-01', ['DAYS_EMPLOYED_DATE_month', 'DAYS_EMPLOYED_DATE_week', 'DAYS_EMPLOYED_DATE_day', 'DAYS_EMPLOYED_DATE_weekday', 'employment_days']] = -1

    df['begin_month_year'] = df['begin_month_DATE'].dt.year
    df['begin_month_month'] = df['begin_month_DATE'].dt.month
    
def create_features(train, test, uid, feature, aggs):
    tr, te = train.copy(), test.copy()
    
    if len(uid)==3:
        uid1, uid2, uid3 = uid[0], uid[1], uid[2]
        tr['uid'] = tr[uid1].astype(str) + '_' + tr[uid2].astype(str) + '_' + tr[uid3].astype(str)
        te['uid'] = te[uid1].astype(str) + '_' + te[uid2].astype(str) + '_' + te[uid3].astype(str)

        for agg in aggs:
            new_col = f'{uid1}_{uid2}_{uid3}_{feature}_{agg}'
            tr[new_col] = tr['uid'].map(tr.groupby('uid')[feature].agg(agg))
            te[new_col] = te['uid'].map(tr.groupby('uid')[feature].agg(agg))
    
    elif len(uid)==2:
        uid1, uid2 = uid[0], uid[1]
        tr['uid'] = tr[uid1].astype(str) + '_' + tr[uid2].astype(str)
        te['uid'] = te[uid1].astype(str) + '_' + te[uid2].astype(str)

        for agg in aggs:
            new_col = f'{uid1}_{uid2}_{feature}_{agg}'
            tr[new_col] = tr['uid'].map(tr.groupby('uid')[feature].agg(agg))
            te[new_col] = te['uid'].map(tr.groupby('uid')[feature].agg(agg))
    else:
        uid1 = uid[0]
        tr['uid'] = tr[uid1].astype(str) + '_'
        te['uid'] = te[uid1].astype(str) + '_'

        for agg in aggs:
            new_col = f'{uid1}__{feature}_{agg}'
            tr[new_col] = tr['uid'].map(tr.groupby('uid')[feature].agg(agg))
            te[new_col] = te['uid'].map(tr.groupby('uid')[feature].agg(agg))

    tr = tr.drop(columns='uid')
    te = te.drop(columns='uid')

    return tr, te

train, test = create_features(train, test, ['DAYS_BIRTH', 'DAYS_EMPLOYED'], 'begin_month', ['mean', 'std', 'max'])
train, test = create_features(train, test, ['DAYS_BIRTH', 'DAYS_EMPLOYED'], 'begin_month_year', ['mean', 'std'])
train, test = create_features(train, test, ['DAYS_BIRTH', 'DAYS_EMPLOYED'], 'begin_month_month', ['mean', 'std'])

# train, test = create_features(train, test, ['cluster'], 'begin_month', ['mean'])
# train, test = create_features(train, test, ['occyp_type'], 'income_total', ['mean'])
# train, test = create_features(train, test, ['cluster'], 'income_total', ['mean'])

train = train.fillna(-99)
test = test.fillna(-99)

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.cluster import KMeans, AgglomerativeClustering

cluster_train = df_train.copy()
cluster_test = df_test.copy()
for col in ['gender', 'car', 'reality', 'edu_type', 'house_type', 'occyp_type', 'income_type', 'family_type', 'work_phone', 'phone', 'email']:
    cluster_train = pd.concat([cluster_train, pd.get_dummies(cluster_train[col], prefix=col)], 1)
    cluster_test = pd.concat([cluster_test, pd.get_dummies(cluster_test[col], prefix=col)], 1)
    cluster_train = cluster_train.drop(columns=col)
    cluster_test = cluster_test.drop(columns=col)

cluster_train.drop(columns='credit', inplace=True)
cluster_train.drop(columns=['index', 'rank', 'uid_rows'], inplace=True)
cluster_test.drop(columns=['index', 'rank', 'uid_rows'], inplace=True)

scaler = MinMaxScaler()
scaler.fit(cluster_train.values)

cluster_train = scaler.transform(cluster_train.values)
cluster_test = scaler.transform(cluster_test.values)

km = KMeans(n_clusters=100, random_state=42, max_iter=1000, )
# km = AgglomerativeClustering(n_clusters=100, )

km.fit(cluster_train)

km_train = km.predict(cluster_train)
km_test = km.predict(cluster_test)

train['cluster'] = km_train
test['cluster'] = km_test

train, test = create_features(train, test, ['cluster'], 'begin_month', ['mean'])
# train, test = create_features(train, test, ['cluster'], 'income_total_y', ['mean'])

drop_col = ['index', 'FLAG_MOBIL', 'DAYS_BIRTH_DATE', 'DAYS_EMPLOYED_DATE', 'begin_month_DATE', 'DAYS_EMPLOYED_binary', 'DAYS_EMPLOYED', 'DAYS_BIRTH',
            'begin_month_year', 'begin_month_month', 'uid_rows', 'rank', 'income_total_y']

In [132]:
train

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,uid_rows,rank,n_dependents2,uid_rows_vc,DAYS_EMPLOYED_binary,DAYS_BIRTH_DATE,DAYS_EMPLOYED_DATE,begin_month_DATE,age,age2,income_total_y,DAYS_BIRTH_month,DAYS_BIRTH_week,DAYS_BIRTH_day,DAYS_BIRTH_weekday,DAYS_EMPLOYED_DATE_month,DAYS_EMPLOYED_DATE_week,DAYS_EMPLOYED_DATE_day,DAYS_EMPLOYED_DATE_weekday,employment_days,begin_month_year,begin_month_month,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_mean,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_std,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_max,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_year_mean,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_year_std,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_month_mean,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_month_std,cluster,cluster__begin_month_mean,cluster__income_total_y_mean
0,0,0.668897,0.620252,0.326076,0.693200,202500.0,0.234418,0.270703,0.687757,0.030918,-13899,-4709,1,0.775258,0.705749,0.90872,-99.000000,0.533280,-6.0,1.0,F_N_N_0_202500.0_Commercial associate_Higher e...,1.0,2.0,5,False,1981-12-11,2007-02-08,2019-06-30,38,22,406822500.0,12,50,11,4,2,6,8,3,-766,2019,6,-33.200000,19.942417,-6.0,2016.800000,1.483240,5.200000,3.701351,52,-24.634409,3.468579e+08
1,1,0.668897,0.620252,0.673924,0.203576,247500.0,0.234418,0.680160,0.080243,0.894017,-11380,-1540,1,0.775258,0.705749,0.09128,0.246746,0.175152,-5.0,1.0,F_N_Y_1_247500.0_Commercial associate_Secondar...,1.0,2.0,9,False,1988-11-03,2015-10-13,2019-07-31,31,29,501187500.0,11,44,3,3,10,42,13,1,-820,2019,7,-7.444444,3.609401,-3.0,2018.888889,0.333333,5.888889,3.515837,13,-22.648464,3.939251e+08
2,2,0.331103,0.379748,0.673924,0.693200,450000.0,0.515743,0.270703,0.687757,0.894017,-19087,-4434,1,0.775258,0.294251,0.90872,0.118506,0.533280,-22.0,2.0,M_Y_Y_0_450000.0_Working_Higher education_Marr...,1.0,2.0,6,False,1967-09-28,2007-11-10,2018-02-28,52,8,908100000.0,9,39,28,3,11,45,10,5,-1222,2018,2,-28.666667,15.279616,-7.0,2017.166667,1.471960,5.333333,3.777124,12,-29.315789,4.947369e+08
3,3,0.668897,0.620252,0.673924,0.693200,202500.0,0.234418,0.680160,0.687757,0.894017,-15088,-2092,1,0.775258,0.294251,0.90872,0.138849,0.533280,-37.0,0.0,F_N_Y_0_202500.0_Commercial associate_Secondar...,1.0,2.0,2,False,1978-09-09,2014-04-09,2016-11-30,41,19,408645000.0,9,36,9,5,4,15,9,2,-1083,2016,11,-44.500000,10.606602,-37.0,2015.500000,0.707107,9.500000,2.121320,61,-25.365759,4.744517e+08
4,4,0.668897,0.379748,0.673924,0.693200,157500.0,0.081415,0.270703,0.687757,0.894017,-15037,-2105,1,0.775258,0.705749,0.90872,0.118506,0.533280,-26.0,2.0,F_Y_Y_0_157500.0_State servant_Higher educatio...,1.0,2.0,1,False,1978-10-30,2014-03-27,2017-10-31,41,19,317677500.0,10,44,30,0,3,13,27,3,-1078,2017,10,-26.000000,-99.000000,-26.0,2017.000000,-99.000000,10.000000,-99.000000,54,-29.523466,4.454522e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,0.668897,0.620252,0.326076,0.089277,225000.0,0.081415,0.680160,0.687757,0.894017,-12079,-1984,1,0.775258,0.705749,0.90872,0.144701,0.085459,-2.0,1.0,F_N_N_2_225000.0_State servant_Secondary / sec...,5.0,2.0,5,False,1986-12-05,2014-07-26,2019-10-31,33,27,454725000.0,12,49,5,4,7,30,26,5,-842,2019,10,-23.600000,19.112823,-2.0,2017.400000,1.516575,7.600000,2.302173,97,-24.280255,3.242160e+08
26453,26453,0.668897,0.620252,0.673924,0.203576,180000.0,0.515743,0.270703,0.058170,0.894017,-15291,-2475,1,0.775258,0.705749,0.90872,-99.000000,0.533280,-47.0,2.0,F_N_Y_1_180000.0_Working_Higher education_Sepa...,1.0,1.0,1,False,1978-02-18,2013-03-22,2016-01-31,41,19,362880000.0,2,7,18,5,3,12,22,4,-1068,2016,1,-47.000000,-99.000000,-47.0,2016.000000,-99.000000,1.000000,-99.000000,30,-24.847775,3.562711e+08
26454,26454,0.668897,0.379748,0.326076,0.693200,292500.0,0.515743,

## rf, xgb

In [ ]:
from sklearn.ensemble import RandomForestClassifier
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

X_train = train.drop(columns=drop_col + [TARGET])
X_test = test.drop(columns=drop_col)
y_train = train[TARGET].values

oof = np.zeros([len(y_train), 3])

clf = RandomForestClassifier(
                    )
clf.fit(X_train, y_train)

oof = clf.predict_proba(X_train)
pred = clf.predict_proba(X_test)
log_loss(y_train, oof)

# for trn_idx, val_idx in skf.split(train, train['credit']):
    
#     clf = RandomForestClassifier(
#                         )
#     clf.fit(X_train[trn_idx], y_train[trn_idx])

#     oof[val_idx] = clf.predict_proba(X_train[val_idx])
#     pred = clf.predict_proba(X_test)

0.17592569278611125

In [ ]:
sub.loc[:, '0':] = pred
sub.to_csv('submission_baseline_rf.csv', index=False)
sub

,index,0,1,2
0,26457,0.040000,0.286667,0.673333
1,26458,0.636667,0.210000,0.153333
2,26459,0.020000,0.020000,0.960000
3,26460,0.040000,0.040000,0.920000
4,26461,0.056667,0.230000,0.713333
...,...,...,...,...
9995,36452,0.100000,0.357667,0.542333
9996,36453,0.230000,0.620000,0.150000
9997,36454,0.000000,0.070000,0.930000
9998,36455,0.540000,0.310000,0.150000


In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
oof = np.zeros([len(train), 3])
pred = np.zeros([len(test), 3])
features = [col for col in test.columns if col not in drop_col]

for trn_idx, val_idx in skf.split(train, train['credit']):
    X_train = train[features]
    X_test = test[features]
    y_train = train[TARGET].values
    
    clf = XGBClassifier(
                         objective='multi:softprob', 
                         num_class=3,
                         metrics='mlogloss',
                         n_estimators=10000,
                         max_depth=8,
                         learning_rate=0.03,
                         colsample_bytree=0.5,
                         subsample=0.7,
                         num_leaves=256,
                         reg_alpha=0.01,
                         reg_lambda=0.01,
                         random_state=0,
                        )
    
    evals = [(X_train.loc[trn_idx], y_train[trn_idx]), (X_train.loc[val_idx], y_train[val_idx])]
    clf.fit(X_train.loc[trn_idx], y_train[trn_idx], eval_set=evals, eval_metric='mlogloss', early_stopping_rounds=50, verbose=1000)

    oof[val_idx] = clf.predict_proba(X_train.loc[val_idx])
    pred += clf.predict_proba(X_test) / 5

[0]	validation_0-mlogloss:1.08448	validation_1-mlogloss:1.08579
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[431]	validation_0-mlogloss:0.412889	validation_1-mlogloss:0.700431

[0]	validation_0-mlogloss:1.08478	validation_1-mlogloss:1.08579
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[466]	validation_0-mlogloss:0.401535	validation_1-mlogloss:0.683297

[0]	validation_0-mlogloss:1.08411	validation_1-mlogloss:1.08487
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[511]	validation_0-mlogloss:0.386507	validation_1-mlogloss:0.67308

[0]	validation_0-mloglo

In [ ]:
oof_xgb = oof.copy()
pred_xgb = pred.copy()

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

target_len = train[TARGET].nunique()

skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
oof = np.zeros([len(train), target_len])
pred = np.zeros([len(test), target_len])

features = [col for col in test.columns if col not in drop_col]

scaler = MinMaxScaler()
scaler.fit(train[features])

for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['credit'])):
    X_train = scaler.transform(train[features])
    X_test = scaler.transform(test[features])
    y_train = train[TARGET].values

    clf = MLPClassifier(hidden_layer_sizes=(4096, 4), max_iter=2000, warm_start=True, momentum=0.01, early_stopping=True,
                         random_state=0,
                        )
    clf.fit(X_train[trn_idx], y_train[trn_idx])

    oof[val_idx] = clf.predict_proba(X_train[val_idx])
    pred += clf.predict_proba(X_test) / 5
    print(idx, 'fold complete ---\n')

log_loss(y_train, oof)

0 fold complete ---

1 fold complete ---

2 fold complete ---

3 fold complete ---

4 fold complete ---



0.8277012703159363

In [ ]:
oof_nn = oof.copy()
pred_nn = pred.copy()

log_loss(y_train, oof)

0.8277012703159363

## lgb

In [133]:
folds = 10

skf = StratifiedKFold(n_splits=folds, random_state=0, shuffle=True)
gk = GroupKFold(n_splits=folds)
kf = KFold(n_splits=folds, random_state=0, shuffle=True)

oof = np.zeros([len(train), 3])
pred = np.zeros([len(test), 3])
lgb_models = {}

params = {'dart':1000, 'gbdt':5000, 'rf':5000, 'goss':5000}
learn_type = 'gbdt'

features = [col for col in test.columns if col not in drop_col]

for idx, (trn_idx, val_idx) in enumerate(gk.split(train, groups=train['rank'])):
    X_train = train[features]
    X_test = test[features]
    y_train = train[TARGET].values

    # print(train.loc[val_idx]['rank'].value_counts())

    clf = LGBMClassifier(
                         boosting_type=learn_type,
                         objective='multiclass', #['binary', 'multiclass', 'multiclassova']
                         num_calsss=3, 
                         metrics='multi_logloss', #['log_loss', 'multi_logloss', 'multi_error']
                         n_estimators=params[learn_type],
                         max_depth=32,
                         learning_rate=0.03,
                         colsample_bytree=0.5,
                         subsample=0.7,
                         num_leaves=256,
                         reg_alpha=0.01,
                         reg_lambda=0.01,
                         random_state=0,
                        )
    evals = [(X_train.loc[trn_idx], y_train[trn_idx]), (X_train.loc[val_idx], y_train[val_idx])]
    clf.fit(X_train.loc[trn_idx], y_train[trn_idx], eval_set=evals, early_stopping_rounds=100, verbose=500)

    if idx==0: feature_importances = clf.feature_importances_ / folds
    else: feature_importances += clf.feature_importances_ / folds

    oof[val_idx] = clf.predict_proba(X_train.loc[val_idx])
    pred += clf.predict_proba(X_test) / folds

    lgb_models[idx] = clf
    print(idx, 'fold complete ################################\n')

log_loss(y_train, oof)

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[164]	valid_0's multi_logloss: 0.395454	valid_1's multi_logloss: 0.680959
0 fold complete ################################

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[207]	valid_0's multi_logloss: 0.367038	valid_1's multi_logloss: 0.651259
1 fold complete ################################

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[214]	valid_0's multi_logloss: 0.373057	valid_1's multi_logloss: 0.623471
2 fold complete ################################

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[207]	valid_0's multi_logloss: 0.381406	valid_1's multi_logloss: 0.635301
3 fold complete ################################

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[204]	valid_0'

0.6681579005710726

In [185]:
# 'category' - reality, family_type
# 5fold best : 0.6714943537141546
# 10fold best : 0.6687885953231191
# add 100kmeans / add 100kmeans gb bm / add 100kmeans gb bmy / add 100kmeans gb bmm : 0.6678719604994495 / 0.6672104903128505 / 0.66793124490285 / 0.6684939580526221
log_loss(y_train, oof)

0.6714505243800835

In [256]:
log_loss(y_train, oof)

0.6709717662626962

In [37]:
oof1 = oof.copy()
pred1 = pred.copy()

In [38]:
log_loss(df_train[TARGET], oof1), log_loss(df_train[TARGET], oof2), log_loss(df_train[TARGET], oof3)

(0.6825658138901987, 0.6857938335701503, 0.6687885953231191)

In [30]:
log_loss(df_train[TARGET], (oof1*.5 + oof2*.5)*.3 + oof3*.7)

0.6672362650866234

In [681]:
log_loss(df_train[TARGET], (oof*.5 + oof1*.5)*.5 + oof2*.5)

0.6685923056479545

In [675]:
log_loss(df_train[TARGET], oof), log_loss(df_train[TARGET], oof1), log_loss(df_train[TARGET], oof2)

(0.6825658138901987, 0.6857938335701503, 0.6716836691352126)

In [673]:
oof2 = oof.copy()

In [666]:
oof1 = oof.copy()

In [ ]:
oof_ = oof.copy()
# oof_[train['rank']>35, 1] = oof_[train['rank']>35, 1] + oof_[train['rank']>35, 0]
oof_[train['rank']>20, 1] = 0.6
oof_[train['rank']>20, 2] = 0.4

log_loss(df_train[TARGET], oof_)

0.6726469575617368

In [105]:
pd.DataFrame({'value':feature_importances, 
              'feature':test[features].columns}).sort_values('value', ascending=False)

,value,feature
14,8844.5,begin_month
29,8157.6,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_mean
35,7927.0,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_month_std
30,7318.3,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_std
15,7306.9,cluster
34,7306.8,DAYS_BIRTH_DAYS_EMPLOYED_begin_month_month_mean
28,7197.3,employment_days
36,6778.6,cluster__begin_month_mean
4,6516.7,income_total
21,6449.8,DAYS_BIRTH_week


In [ ]:
log_loss(df_train[TARGET], (oof1*.5 + oof2*.5)*.3 + oof3*.7)

In [278]:
sub.loc[:, '0':] = pred
sub.to_csv('submission_8folds_gk.csv', index=False)
sub

,index,0,1,2
0,26457,0.066171,0.150268,0.783561
1,26458,0.353528,0.284718,0.361753
2,26459,0.027714,0.056465,0.915821
3,26460,0.069056,0.065401,0.865544
4,26461,0.071339,0.251301,0.677360
...,...,...,...,...
9995,36452,0.071514,0.217797,0.710689
9996,36453,0.266893,0.270864,0.462243
9997,36454,0.035259,0.106674,0.858068
9998,36455,0.156627,0.342894,0.500478


In [ ]:
sub.loc[:, '0':] = pred
sub.to_csv('submission_10folds_lgb.csv', index=False)
sub

,index,0,1,2
0,26457,0.131618,0.363444,1.504938
1,26458,0.774156,0.567124,0.658720
2,26459,0.042190,0.144810,1.812999
3,26460,0.100001,0.118077,1.781922
4,26461,0.133720,0.557447,1.308832
...,...,...,...,...
9995,36452,0.208955,0.497983,1.293061
9996,36453,0.569811,0.533473,0.896716
9997,36454,0.073159,0.237876,1.688965
9998,36455,0.397758,0.733316,0.868926


In [ ]:
# df_train['rank'] = 0
# temp_dict = {}
# for i in df_train.index:

#     key = '_'.join(df_train.loc[i, :'family_size'].astype(str))
#     try:
#         temp_dict[key]
#         temp_dict[key] += 1
#     except:
#         temp_dict[key] = 0
    
#     df_train.loc[i, 'rank'] = temp_dict[key]
# df_test['rank']=0

# df_train['duplicated'] = df_train.loc[:, :'begin_month'].duplicated().astype(int)
# df_test['duplicated'] = df_test.loc[:, :'begin_month'].duplicated().astype(int)

In [ ]:
test = train[22000:].reset_index(drop=True)
train = train[:22000].reset_index(drop=True)

In [ ]:
test = test.drop(columns='credit')

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

oof = np.zeros([len(train), 2])
pred = np.zeros([len(test), 2])
lgb_models = {}

params = {'dart':100, 'gbdt':5000, 'rf':5000, 'goss':5000}
learn_type = 'gbdt'

features = [col for col in test.columns if col not in drop_col]

for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['credit'])):
    X_train = train[features]
    X_test = test[features]
    y_train = train[TARGET].map({0:0, 1:1, 2:1}).values

    clf = LGBMClassifier(
                         boosting_type=learn_type,
                         objective='binary', #['binary', 'multiclass', 'multiclassova']
                         metrics='log_loss', #['log_loss', 'multi_logloss', 'multi_error']
                         n_estimators=params[learn_type],
                         max_depth=64,
                         learning_rate=0.03,
                         colsample_bytree=0.5,
                         subsample=0.7,
                         num_leaves=256,
                         reg_alpha=0.01,
                         reg_lambda=0.01,
                         random_state=0,
                        )
    evals = [(X_train.loc[trn_idx], y_train[trn_idx]), (X_train.loc[val_idx], y_train[val_idx])]
    clf.fit(X_train.loc[trn_idx], y_train[trn_idx], eval_set=evals, eval_metric='logloss', early_stopping_rounds=100, verbose=500)

    if idx==0: feature_importances = clf.feature_importances_/5
    else: feature_importances += clf.feature_importances_/5

    oof[val_idx] = clf.predict_proba(X_train.loc[val_idx], num_iteration=clf.best_iteration_)
    pred += clf.predict_proba(X_test, num_iteration=clf.best_iteration_) / 5

    lgb_models[idx] = clf
    print(idx, 'fold complete ################################\n')

log_loss(y_train, oof)

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[82]	valid_0's binary_logloss: 0.196579	valid_1's binary_logloss: 0.31414
0 fold complete ################################

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[101]	valid_0's binary_logloss: 0.179188	valid_1's binary_logloss: 0.312159
1 fold complete ################################

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[115]	valid_0's binary_logloss: 0.167826	valid_1's binary_logloss: 0.309396
2 fold complete ################################

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[72]	valid_0's binary_logloss: 0.206537	valid_1's binary_logloss: 0.325476
3 fold complete ################################

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[88]	vali

0.31699643690860324

In [ ]:
train = pd.concat([train, pd.DataFrame(oof, columns=['p1', 'p2'])], 1)
test = pd.concat([test, pd.DataFrame(pred, columns=['p1', 'p2'])], 1)

In [ ]:
# train['credit'] = train['credit'].map({0:0, 1:1, 2:1})
# train = train[train[TARGET] != -1].reset_index(drop=True)

target_len = train[TARGET].nunique()
folds = 5

skf = StratifiedKFold(n_splits=folds, random_state=0, shuffle=True)
gk = GroupKFold(n_splits=folds)
kf = KFold(n_splits=folds, random_state=0, shuffle=True)

oof = np.zeros([len(train), target_len])
pred = np.zeros([len(test), target_len])
lgb_models = {}

# y = train['begin_month'].astype(str) + '_' +train['credit'].astype(str)
# for trn_idx, val_idx in skf.split(train, y):

params = {'dart':1000, 'gbdt':5000, 'rf':5000, 'goss':5000}
learn_type = 'gbdt'

features = [col for col in test.columns if col not in drop_col]

# for idx, (trn_idx, val_idx) in enumerate(kf.split(train)):
# for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['credit'].map({0:0, 1:1, 2:1}))):
# for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['rank'])):
# for idx, (trn_idx, val_idx) in enumerate(gk.split(train, groups=train['prior_credit'])):
# for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['begin_month'].astype(str) + '_' + train['credit'].astype(str))):
# for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['credit'])):
for idx, (trn_idx, val_idx) in enumerate(gk.split(train, groups=train['rank'])):
    X_train = train[features]
    X_test = test[features]
    y_train = train[TARGET].values

    # print(train.loc[val_idx]['rank'].value_counts())

    clf = LGBMClassifier(
                         boosting_type=learn_type,
                         objective='multiclass', #['binary', 'multiclass', 'multiclassova']
                         num_calsss=3, 
                         metrics='multi_logloss', #['log_loss', 'multi_logloss', 'multi_error']
                         n_estimators=params[learn_type],
                         max_depth=32,
                         learning_rate=0.03,
                         colsample_bytree=0.5,
                         subsample=0.7,
                         num_leaves=256,
                         reg_alpha=0.01,
                         reg_lambda=0.01,
                         random_state=0,
                        )
    evals = [(X_train.loc[trn_idx], y_train[trn_idx]), (X_train.loc[val_idx], y_train[val_idx])]
    clf.fit(X_train.loc[trn_idx], y_train[trn_idx], eval_set=evals, early_stopping_rounds=100, verbose=500)

    if idx==0: feature_importances = clf.feature_importances_ / folds
    else: feature_importances += clf.feature_importances_ / folds

    oof[val_idx] = clf.predict_proba(X_train.loc[val_idx])
    pred += clf.predict_proba(X_test) / folds

    lgb_models[idx] = clf
    print(idx, 'fold complete ################################\n')

log_loss(y_train, oof)

In [ ]:
# train['credit'] = train['credit'].map({0:0, 1:1, 2:1})
# train = train[train[TARGET] != -1].reset_index(drop=True)

target_len = train[TARGET].nunique()

skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
gk = GroupKFold(n_splits=5)
kf = KFold(n_splits=5, random_state=0, shuffle=True)

oof = np.zeros([len(train), target_len])
pred = np.zeros([len(test), target_len])
lgb_models = {}

# y = train['begin_month'].astype(str) + '_' +train['credit'].astype(str)
# for trn_idx, val_idx in skf.split(train, y):

params = {'dart':100, 'gbdt':5000, 'rf':5000, 'goss':5000}
learn_type = 'gbdt'

features = [col for col in test.columns if col not in drop_col]

# for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['begin_month'].astype(str) + '_' + train['credit'].astype(str))):
# for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['credit'].map({0:0, 1:1, 2:1}))):
# for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['credit'])):
# for idx, (trn_idx, val_idx) in enumerate(skf.split(train, train['rank'])):
# for idx, (trn_idx, val_idx) in enumerate(kf.split(train)):
for idx, (trn_idx, val_idx) in enumerate(gk.split(train, groups=train['rank'])):
    X_train = train[features]
    X_test = test[features]
    y_train = train[TARGET].values

    print(train.loc[val_idx]['rank'].value_counts())

    clf = LGBMClassifier(
                         boosting_type=learn_type,
                         objective='multiclass', #['binary', 'multiclass', 'multiclassova']
                         num_calsss=3, 
                         metrics='multi_logloss', #['log_loss', 'multi_logloss', 'multi_error']
                         n_estimators=params[learn_type],
                         max_depth=64,
                         learning_rate=0.03,
                         colsample_bytree=0.5,
                         subsample=0.7,
                         num_leaves=256,
                         reg_alpha=0.01,
                         reg_lambda=0.01,
                         random_state=0,
                        )
    evals = [(X_train.loc[trn_idx], y_train[trn_idx]), (X_train.loc[val_idx], y_train[val_idx])]
    clf.fit(X_train.loc[trn_idx], y_train[trn_idx], eval_set=evals, early_stopping_rounds=100, verbose=500)

    if idx==0: feature_importances = clf.feature_importances_/5
    else: feature_importances += clf.feature_importances_/5

    oof[val_idx] = clf.predict_proba(X_train.loc[val_idx], num_iteration=clf.best_iteration_)
    pred += clf.predict_proba(X_test, num_iteration=clf.best_iteration_) / 5

    lgb_models[idx] = clf
    print(idx, 'fold complete ################################\n')

log_loss(y_train, oof)

In [ ]:
oof_0 = oof[:, 0]
pred_0 = pred[:, 0]

In [ ]:
oof_temp = oof.copy()
pred_temp = pred.copy()

In [ ]:
oof_temp[:, 0] = oof_0
pred_temp[:, 0] = pred_0

In [ ]:
log_loss(df_train[TARGET], oof_temp)

0.6815677554297949

In [ ]:
oof_ = np.zeros([len(train), target_len])

for iter, (i,j,k) in enumerate(oof_temp):
    if sum([i,j,k]) < 1:
        d = (1 - i - j - k)/2
        # j += d
        # k += d
    else:
        d = (i + j + k - 1)/3
        i -= d
        j -= d
        k -= d
        
    oof_[iter] = [i,j,k]
log_loss(df_train[TARGET], oof_)

In [ ]:
# hyper optimization으로 찾아낸 parameter
# lightgbm dart 사용, 보다 lb 0.03 정도 좋음
# gbdt가 0.3285라면 dart는 0.3255, goss는 0.3300
lgb_param_dart = {'objective': 'multiclass', 
 'num_class': 19, 
 'boosting_type': 'dart', 
 'subsample_freq': 5, 
 'num_leaves': 92, 
 'min_data_in_leaf': 64, 
 'subsample_for_bin': 23000, 
 'max_depth': 10, 
 'feature_fraction': 0.302, 
 'bagging_fraction': 0.904, 
 'lambda_l1': 0.099, 
 'lambda_l2': 1.497, 
 'min_child_weight': 38.011, 
 'nthread': 32, 
 'metric': 'multi_logloss', 
 'learning_rate': 0.021, 
 'min_sum_hessian_in_leaf': 3, 
 'drop_rate': 0.846244, 
 'skip_drop': 0.792465, 
 'max_drop': 65,
 'seed': 42,
 'n_estimators': 1000}

In [ ]:
train['phone'] = ((train['phone'] + train['work_phone'])>1).astype(int)
test['phone'] = ((test['phone'] + test['work_phone'])>1).astype(int)

In [ ]:
train['uid'] = (
                train['DAYS_BIRTH'].astype(str) + '_' + 
                # train['car'].astype(str) + '_' + 
                # train['reality'].astype(str) + '_' + 
                # train['child_num'].astype(str) + '_' + 
                # train['income_type'].astype(str) + '_' + 
                # train['edu_type'].astype(str) + '_' + 
                train['DAYS_EMPLOYED'].astype(str))

test['uid'] = (
                test['DAYS_BIRTH'].astype(str) + '_' + 
                # test['car'].astype(str) + '_' + 
                # test['reality'].astype(str) + '_' + 
                # test['child_num'].astype(str) + '_' + 
                # test['income_type'].astype(str) + '_' + 
                # test['edu_type'].astype(str) + '_' + 
                test['DAYS_EMPLOYED'].astype(str))

train['ttt'] = train['uid'].map(train.groupby('uid')['begin_month'].mean()).fillna(-99)
test['ttt'] = test['uid'].map(train.groupby('uid')['begin_month'].mean()).fillna(-99)
train['tttt'] = train['uid'].map(train.groupby('uid')['begin_month'].std()).fillna(-99)
test['tttt'] = test['uid'].map(train.groupby('uid')['begin_month'].std()).fillna(-99)

# train['uid'] = (
#                 train['DAYS_BIRTH'].astype(str) + '_' + 
#                 # train['gender'].astype(str) + '_' + 
#                 # train['reality'].astype(str) + '_' + 
#                 # train['child_num'].astype(str) + '_' + 
#                 train['income_type'].astype(str) + '_' + 
#                 # train['edu_type'].astype(str) + '_' + 
#                 train['DAYS_EMPLOYED'].astype(str))

# test['uid'] = (
#                 test['DAYS_BIRTH'].astype(str) + '_' + 
#                 # test['gender'].astype(str) + '_' + 
#                 # test['reality'].astype(str) + '_' + 
#                 # test['child_num'].astype(str) + '_' + 
#                 test['income_type'].astype(str) + '_' + 
#                 # test['edu_type'].astype(str) + '_' + 
#                 test['DAYS_EMPLOYED'].astype(str))

# train['attt'] = train['uid'].map(train.groupby('DAYS_BIRTH')['begin_month'].mean())
# test['attt'] = test['uid'].map(train.groupby('DAYS_BIRTH')['begin_month'].mean()).fillna(-99)
# train['atttt'] = train['uid'].map(train.groupby('DAYS_BIRTH')['begin_month'].std())
# test['atttt'] = test['uid'].map(train.groupby('DAYS_BIRTH')['begin_month'].std()).fillna(-99)

# train['bttt'] = train['DAYS_EMPLOYED'].map(train.groupby('DAYS_EMPLOYED')['begin_month'].mean())
# test['bttt'] = test['DAYS_EMPLOYED'].map(train.groupby('DAYS_EMPLOYED')['begin_month'].mean()).fillna(-99)
# train['btttt'] = train['DAYS_EMPLOYED'].map(train.groupby('DAYS_EMPLOYED')['begin_month'].std())
# test['btttt'] = test['DAYS_EMPLOYED'].map(train.groupby('DAYS_EMPLOYED')['begin_month'].std()).fillna(-99) 

In [ ]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
# from sklearn.cluster import KMeans, AgglomerativeClustering

# cluster_train = df_train.copy()
# cluster_test = df_test.copy()
# for col in ['gender', 'car', 'reality', 'edu_type', 'house_type', 'occyp_type', 'income_type', 'family_type', 'work_phone', 'phone', 'email']:
#     cluster_train = pd.concat([cluster_train, pd.get_dummies(cluster_train[col], prefix=col)], 1)
#     cluster_test = pd.concat([cluster_test, pd.get_dummies(cluster_test[col], prefix=col)], 1)
#     cluster_train = cluster_train.drop(columns=col)
#     cluster_test = cluster_test.drop(columns=col)
# cluster_train.drop(columns='credit', inplace=True)
# # d = [col for col in cluster_train.columns if 'DATE' not in col]
# # cluster_train = cluster_train[d]
# # cluster_test = cluster_test[d]

# cluster_train = cluster_train[['DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month']]
# cluster_test = cluster_test[['DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month']]


# scaler = MinMaxScaler()
# scaler.fit(cluster_train.values)

# cluster_train = scaler.transform(cluster_train.values)
# cluster_test = scaler.transform(cluster_test.values)

# km = KMeans(n_clusters=100, random_state=42, max_iter=1000, )
# # km = AgglomerativeClustering(n_clusters=100, )

# km.fit(cluster_train)

# km_train = km.predict(cluster_train)
# km_test = km.predict(cluster_test)

# train['cluster'] = km_train
# test['cluster'] = km_test

In [ ]:
# train = pd.concat([train, pd.get_dummies(train['child_num'], prefix='child_num')], 1)
# test = pd.concat([test, pd.get_dummies(test['child_num'], prefix='child_num')], 1)

# train = pd.concat([train, pd.get_dummies(train['begin_month'], prefix='begin_month')], 1)
# test = pd.concat([test, pd.get_dummies(test['begin_month'], prefix='begin_month')], 1)

# train = pd.concat([train, pd.get_dummies(train['edu_type'], prefix='edu_type')], 1)
# test = pd.concat([test, pd.get_dummies(test['edu_type'], prefix='edu_type')], 1)

# train = pd.concat([train, pd.get_dummies(train['house_type'], prefix='house_type')], 1)
# test = pd.concat([test, pd.get_dummies(test['house_type'], prefix='house_type')], 1)

# train = pd.concat([train, pd.get_dummies(train['occyp_type'], prefix='occyp_type')], 1)
# test = pd.concat([test, pd.get_dummies(test['occyp_type'], prefix='occyp_type')], 1)

# train = pd.concat([train, pd.get_dummies(train['income_type'], prefix='income_type')], 1)
# test = pd.concat([test, pd.get_dummies(test['income_type'], prefix='income_type')], 1)

# train = pd.concat([train, pd.get_dummies(train['family_type'], prefix='family_type')], 1)
# test = pd.concat([test, pd.get_dummies(test['family_type'], prefix='family_type')], 1)

In [ ]:
    # t_features = ['t1', 't2', 't3']
    # a = train.loc[trn_idx].groupby('DAYS_BIRTH')[TARGET].value_counts(True).unstack().fillna(0.0)
    # a.columns = t_features

    # for col in t_features:
    #     X_train = np.concatenate([X_train, train['DAYS_BIRTH'].map(a[col].to_dict()).values.reshape(-1, 1)], 1)
    #     X_test = np.concatenate([X_test, test['DAYS_BIRTH'].map(a[col].to_dict()).values.reshape(-1, 1)], 1)

    # t_features = ['t4', 't5', 't6']
    # a = train.loc[trn_idx].groupby('begin_month')[TARGET].value_counts(True).unstack().fillna(0.0)
    # a.columns = t_features

    # for col in t_features:
    #     X_train = np.concatenate([X_train, train['begin_month'].map(a[col].to_dict()).values.reshape(-1, 1)], 1)
    #     X_test = np.concatenate([X_test, test['begin_month'].map(a[col].to_dict()).values.reshape(-1, 1)], 1)

    # t_features = ['t7', 't8', 't9']
    # a = train.loc[trn_idx].groupby('DAYS_BIRTH_day')[TARGET].value_counts(True).unstack().fillna(0.0)
    # a.columns = t_features

    # for col in t_features:
    #     X_train = np.concatenate([X_train, train['DAYS_BIRTH_day'].map(a[col].to_dict()).values.reshape(-1, 1)], 1)
    #     X_test = np.concatenate([X_test, test['DAYS_BIRTH_day'].map(a[col].to_dict()).values.reshape(-1, 1)], 1)

    # t_features = ['t10', 't11', 't12']
    # a = train.loc[trn_idx].groupby('DAYS_BIRTH_month')[TARGET].value_counts(True).unstack().fillna(0.0)
    # a.columns = t_features

    # for col in t_features:
    #     X_train = np.concatenate([X_train, train['DAYS_BIRTH_month'].map(a[col].to_dict()).values.reshape(-1, 1)], 1)
    #     X_test = np.concatenate([X_test, test['DAYS_BIRTH_month'].map(a[col].to_dict()).values.reshape(-1, 1)], 1)

In [ ]:
# pp를 해봄

# adj_arr = []
# for i in range(oof.shape[0]):
#     temp = oof[i].copy()
#     if (oof[i]<0.1).any():
#         min = oof[i][np.argmin(oof[i])]

#         temp[np.argmin(temp)] = temp[np.argmin(temp)] - min
#         temp[np.argmax(temp)] = temp[np.argmax(temp)] + min

#     adj_arr += [temp]
# adj_arr = np.array(adj_arr)
# log_loss(y_train, adj_arr)

In [ ]:
# train['edu_type'] = train['edu_type'].astype('category')
# test['edu_type'] = test['edu_type'].astype('category')

# train['income_type'] = train['income_type'].astype('category')
# test['income_type'] = test['income_type'].astype('category')

# train['family_type'] = train['family_type'].astype('category')
# test['family_type'] = test['family_type'].astype('category')

# train['house_type'] = train['house_type'].astype('category')
# test['house_type'] = test['house_type'].astype('category')

In [ ]:
# train['DAYS_EMPLOYED_day'] = (train['DAYS_EMPLOYED']/24).round()
# test['DAYS_EMPLOYED_day'] = (test['DAYS_EMPLOYED']/24).round()

# train['phone2'] = train[['work_phone', 'phone']].sum(1)
# test['phone2'] = test[['work_phone', 'phone']].sum(1)

# col = 'DAYS_EMPLOYED'
# train['income_mean'] = train[col].map(train.groupby(col)['income_total'].mean().to_dict()).round(2)
# test['income_mean'] = test[col].map(train.groupby(col)['income_total'].mean().to_dict()).round(2)

# col1 = 'DAYS_BIRTH_month'
# col2 = 'DAYS_BIRTH_day'
# for df in [train, test]:
#     df['ttt'] = df[col1].astype(str) + df[col2].astype(str)
# test['ttt'] = test['ttt'].map(train['ttt'].value_counts())
# train['ttt'] = train['ttt'].map(train['ttt'].value_counts())



# train['bigin_year'] = (train['begin_month']/12).round()
# test['bigin_year'] = (test['begin_month']/12).round()

# date_dict = {i:j for i,j in zip(np.unique(train['begin_month']), pd.date_range('2014-01-01', periods=60, freq='M'))}
# train['begin_month_date'] = train['begin_month'].map(date_dict)
# test['begin_month_date'] = test['begin_month'].map(date_dict)

# train['begin_month_date_month'] = train['begin_month_date'].dt.month
# test['begin_month_date_month'] = test['begin_month_date'].dt.month

In [ ]:
/usr/local/lib/python3.7/dist-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')
[100]	valid_0's multi_logloss: 0.717462	valid_1's multi_logloss: 0.795645
[200]	valid_0's multi_logloss: 0.630438	valid_1's multi_logloss: 0.748241
[300]	valid_0's multi_logloss: 0.569636	valid_1's multi_logloss: 0.72224
[400]	valid_0's multi_logloss: 0.521711	valid_1's multi_logloss: 0.703517
[500]	valid_0's multi_logloss: 0.487992	valid_1's multi_logloss: 0.696224
[600]	valid_0's multi_logloss: 0.456908	valid_1's multi_logloss: 0.689985
[700]	valid_0's multi_logloss: 0.419924	valid_1's multi_logloss: 0.683839
[800]	valid_0's multi_logloss: 0.394576	valid_1's multi_logloss: 0.682825
[900]	valid_0's multi_logloss: 0.358987	valid_1's multi_logloss: 0.682866
[1000]	valid_0's multi_logloss: 0.332996	valid_1's multi_logloss: 0.685887
0 fold complete ################################

[100]	valid_0's multi_logloss: 0.725321	valid_1's multi_logloss: 0.790456
[200]	valid_0's multi_logloss: 0.643862	valid_1's multi_logloss: 0.739061
[300]	valid_0's multi_logloss: 0.586349	valid_1's multi_logloss: 0.707981
[400]	valid_0's multi_logloss: 0.53976	valid_1's multi_logloss: 0.686057
[500]	valid_0's multi_logloss: 0.507105	valid_1's multi_logloss: 0.676175
[600]	valid_0's multi_logloss: 0.476674	valid_1's multi_logloss: 0.667773
[700]	valid_0's multi_logloss: 0.439057	valid_1's multi_logloss: 0.658935
[800]	valid_0's multi_logloss: 0.413621	valid_1's multi_logloss: 0.654829
[900]	valid_0's multi_logloss: 0.378107	valid_1's multi_logloss: 0.651918
[1000]	valid_0's multi_logloss: 0.351965	valid_1's multi_logloss: 0.652321
1 fold complete ################################

[100]	valid_0's multi_logloss: 0.729532	valid_1's multi_logloss: 0.780185
[200]	valid_0's multi_logloss: 0.648001	valid_1's multi_logloss: 0.726115
[300]	valid_0's multi_logloss: 0.590496	valid_1's multi_logloss: 0.694461
[400]	valid_0's multi_logloss: 0.544351	valid_1's multi_logloss: 0.673411
[500]	valid_0's multi_logloss: 0.512577	valid_1's multi_logloss: 0.663777
[600]	valid_0's multi_logloss: 0.48281	valid_1's multi_logloss: 0.65515
[700]	valid_0's multi_logloss: 0.44577	valid_1's multi_logloss: 0.646621
[800]	valid_0's multi_logloss: 0.420848	valid_1's multi_logloss: 0.64398
[900]	valid_0's multi_logloss: 0.38563	valid_1's multi_logloss: 0.641078
[1000]	valid_0's multi_logloss: 0.359937	valid_1's multi_logloss: 0.640073
2 fold complete ################################

[100]	valid_0's multi_logloss: 0.727141	valid_1's multi_logloss: 0.79466
[200]	valid_0's multi_logloss: 0.644628	valid_1's multi_logloss: 0.74353
[300]	valid_0's multi_logloss: 0.586057	valid_1's multi_logloss: 0.715921
[400]	valid_0's multi_logloss: 0.538595	valid_1's multi_logloss: 0.695579
[500]	valid_0's multi_logloss: 0.506456	valid_1's multi_logloss: 0.686885
[600]	valid_0's multi_logloss: 0.476331	valid_1's multi_logloss: 0.681176
[700]	valid_0's multi_logloss: 0.43846	valid_1's multi_logloss: 0.672951
[800]	valid_0's multi_logloss: 0.413368	valid_1's multi_logloss: 0.670078
[900]	valid_0's multi_logloss: 0.378338	valid_1's multi_logloss: 0.667854
[1000]	valid_0's multi_logloss: 0.352662	valid_1's multi_logloss: 0.667985
3 fold complete ################################

[100]	valid_0's multi_logloss: 0.71988	valid_1's multi_logloss: 0.815058
[200]	valid_0's multi_logloss: 0.637212	valid_1's multi_logloss: 0.770251
[300]	valid_0's multi_logloss: 0.578601	valid_1's multi_logloss: 0.746026
[400]	valid_0's multi_logloss: 0.531626	valid_1's multi_logloss: 0.729811
[500]	valid_0's multi_logloss: 0.499056	valid_1's multi_logloss: 0.723775
[600]	valid_0's multi_logloss: 0.469467	valid_1's multi_logloss: 0.719303
[700]	valid_0's multi_logloss: 0.432082	valid_1's multi_logloss: 0.715957
[800]	valid_0's multi_logloss: 0.407626	valid_1's multi_logloss: 0.714832
[900]	valid_0's multi_logloss: 0.37257	valid_1's multi_logloss: 0.716798
[1000]	valid_0's multi_logloss: 0.346343	valid_1's multi_logloss: 0.719622
4 fold complete ################################

0.6742893830660625

In [ ]:
# df_train = pd.read_csv('open/train.csv')
# df_test = pd.read_csv('open/test.csv')
# sub = pd.read_csv('open/sample_submission.csv')

# # df_train = df_train.sort_values('begin_month', ascending=True).reset_index(drop=True)
# # df_test = df_test.sort_values('begin_month', ascending=True).reset_index(drop=True)

# df_train.loc[df_train['child_num'] > df_train['family_size'], 'family_size'] = 4
# df_train.loc[(df_train['family_type']=='Married') & (df_train['family_size']<2), 'family_size'] = 3
# df_train.loc[df_train['child_num'] == df_train['family_size'], 'family_size'] = 2

# df_test.loc[df_test['child_num'] > df_test['family_size'], 'family_size'] = 4
# df_test.loc[(df_test['family_type']=='Married') & (df_test['family_size']<2), 'family_size'] = 3

# # df_train = df_train.loc[df_train.loc[:, 'gender':'begin_month'].drop_duplicates().index].reset_index(drop=True)
# # df_train = df_train.loc[:, 'gender':'credit'].drop_duplicates().reset_index(drop=True)

# temp = []
# duplicates = []
# for idx in tqdm(range(len(df_train))):
#     row = '_'.join(df_train.loc[idx, 'gender':'family_size'].astype(str).tolist())
#     # row = '_'.join(df_train.loc[idx, ['DAYS_BIRTH', 'DAYS_EMPLOYED']].astype(str).tolist())
#     # row = '_'.join(df_train.loc[idx, 'car':'family_size'].astype(str).tolist())
#     if row in temp:
#         # duplicates.append(np.isin(temp, row).sum() + 1)
#         duplicates.append(sum([row==i for i in temp]) + 1)
#     else:
#         duplicates.append(1)
#     temp.append(row)

# df_train['rank'] = duplicates
# df_train['uid_rows'] = temp

# duplicates = []
# for idx in tqdm(range(len(df_test))):
#     row = '_'.join(df_test.loc[idx, 'gender':'family_size'].astype(str).tolist())
#     if row in temp:
#         # duplicates.append(np.isin(temp, row).sum() + 1)
#         duplicates.append(sum([row==i for i in temp]) + 1)
#     else:
#         duplicates.append(1)
#     temp.append(row)

# df_test['rank'] = duplicates
# df_test['uid_rows'] = temp[-10000:]

train = df_train.copy()
test = df_test.copy()

# 부양가족
# train['n_dependents'] = train['family_size'] - train['family_type'].apply(lambda x: 2 if 'Marr' in x else 1)
# test['n_dependents'] = test['family_size'] - test['family_type'].apply(lambda x: 2 if 'Marr' in x else 1)
train['n_dependents2'] = train['family_size'] - train['child_num']
test['n_dependents2'] = test['family_size'] - test['child_num']
# train['n_dependents3'] = train['n_dependents'] - train['child_num']
# test['n_dependents3'] = test['n_dependents'] - test['child_num']

TARGET = 'credit'

# categorical 
for df in [train, test]:
    df['occyp_type'].fillna('NULL', inplace=True)
    df['reality'] = df['reality'].astype('category')
    df['family_type'] = df['family_type'].astype('category')
    df['gender'] = df['gender'].map({'F':0, 'M':1})

for col in ['car', 'edu_type', 'house_type', 'occyp_type', 'income_type', 'work_phone', 'phone', 'email']:
    temp = train[col].value_counts(True).to_dict()
    for df in [train, test]:
        df[col] = df[col].map(temp)

test['uid_rows_vc'] = test['uid_rows'].map(train['uid_rows'].value_counts())
train['uid_rows_vc'] = train['uid_rows'].map(train['uid_rows'].value_counts())

train['DAYS_EMPLOYED_binary'] = train['DAYS_EMPLOYED']>0
test['DAYS_EMPLOYED_binary'] = test['DAYS_EMPLOYED']>0

train['DAYS_BIRTH_DATE'] = train['DAYS_BIRTH'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=25153, freq='d')[::-1])})
test['DAYS_BIRTH_DATE'] = test['DAYS_BIRTH'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=25153, freq='d')[::-1])})

# 미고용 상태 NULL
train['DAYS_EMPLOYED_DATE'] = train['DAYS_EMPLOYED'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=15714, freq='d')[::-1])})
test['DAYS_EMPLOYED_DATE'] = test['DAYS_EMPLOYED'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=15714, freq='d')[::-1])})
train['DAYS_EMPLOYED_DATE'] = pd.to_datetime(train['DAYS_EMPLOYED_DATE'].fillna('2020-01-01'))
test['DAYS_EMPLOYED_DATE'] = pd.to_datetime(test['DAYS_EMPLOYED_DATE'].fillna('2020-01-01'))


train['begin_month_DATE'] = train['begin_month'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=61, freq='m')[::-1])})
test['begin_month_DATE'] = test['begin_month'].abs().map({k:v for k,v in enumerate(pd.date_range(end='2019-12-31', periods=61, freq='m')[::-1])})

for df in [train, test]:

    # 2019년 기준 현재 나이, 정년까지 남은 연수
    df['age'] = 2019 - df['DAYS_BIRTH_DATE'].dt.year
    df['age2'] = 60 - df['age']
    # 2019년 기준 일한 햇수
    # df['EMPLOYED_year'] = (2019 - df['DAYS_EMPLOYED_DATE'].dt.year)
    # df['age3'] = df['age'] - df['age3']
    # df['age3'] = df['age'] - (2019 - df['begin_month_DATE'].dt.year)
    # df['loan_age'] = df['age'] - (2019 - df['begin_month_year'])
    # df['loan_age'] = df['loan_age']//10

    df['DAYS_BIRTH_week'] = df['DAYS_BIRTH_DATE'].dt.isocalendar().week.astype(int)
    df['DAYS_BIRTH_day'] = df['DAYS_BIRTH_DATE'].dt.day
    df['DAYS_BIRTH_weekday'] = df['DAYS_BIRTH_DATE'].dt.weekday

    df['DAYS_EMPLOYED_10y'] = df['DAYS_EMPLOYED_DATE'].dt.year//10
    df['DAYS_EMPLOYED_DATE_month'] = df['DAYS_EMPLOYED_DATE'].dt.month
    df['DAYS_EMPLOYED_DATE_week'] = df['DAYS_EMPLOYED_DATE'].dt.isocalendar().week.astype(int)
    df['DAYS_EMPLOYED_DATE_day'] = df['DAYS_EMPLOYED_DATE'].dt.day
    df['DAYS_EMPLOYED_DATE_weekday'] = df['DAYS_EMPLOYED_DATE'].dt.weekday

    df['employment_days'] = (df['DAYS_BIRTH_DATE'] - df['DAYS_EMPLOYED_DATE']).dt.days
    df['employment_days'] = df['employment_days']//12

    df.loc[df['DAYS_EMPLOYED_DATE']=='2020-01-01', ['DAYS_EMPLOYED_DATE_month', 'DAYS_EMPLOYED_DATE_week', 'DAYS_EMPLOYED_DATE_day', 'DAYS_EMPLOYED_DATE_weekday', 'employment_days']] = -1

    df['begin_month_year'] = df['begin_month_DATE'].dt.year
    df['begin_month_month'] = df['begin_month_DATE'].dt.month
    
    # df['diff_DAYS_EMPLOYED_begin_month_month'] = (df['begin_month_year']*12 + df['begin_month_month']) - (df['DAYS_EMPLOYED_DATE'].dt.month*12 + df['DAYS_EMPLOYED_DATE_month'])
    # df['diff_DAYS_EMPLOYED_begin_month_month'] = (df['diff_DAYS_EMPLOYED_begin_month_month']/12).round()

# for col in ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'income_total']:
#     for df in [train, test]:
#         df[col] = np.log1p(df[col])

def create_features(train, test, uid, feature, aggs):
    tr, te = train.copy(), test.copy()
    
    if len(uid)==3:
        uid1, uid2, uid3 = uid[0], uid[1], uid[2]
        tr['uid'] = tr[uid1].astype(str) + '_' + tr[uid2].astype(str) + '_' + tr[uid3].astype(str)
        te['uid'] = te[uid1].astype(str) + '_' + te[uid2].astype(str) + '_' + te[uid3].astype(str)

        for agg in aggs:
            new_col = f'{uid1}_{uid2}_{uid3}_{feature}_{agg}'
            tr[new_col] = tr['uid'].map(tr.groupby('uid')[feature].agg(agg))
            te[new_col] = te['uid'].map(tr.groupby('uid')[feature].agg(agg))
    
    elif len(uid)==2:
        uid1, uid2 = uid[0], uid[1]
        tr['uid'] = tr[uid1].astype(str) + '_' + tr[uid2].astype(str)
        te['uid'] = te[uid1].astype(str) + '_' + te[uid2].astype(str)

        for agg in aggs:
            new_col = f'{uid1}_{uid2}_{feature}_{agg}'
            tr[new_col] = tr['uid'].map(tr.groupby('uid')[feature].agg(agg))
            te[new_col] = te['uid'].map(tr.groupby('uid')[feature].agg(agg))
    else:
        uid1 = uid[0]
        tr['uid'] = tr[uid1].astype(str) + '_'
        te['uid'] = te[uid1].astype(str) + '_'

        for agg in aggs:
            new_col = f'{uid1}__{feature}_{agg}'
            tr[new_col] = tr['uid'].map(tr.groupby('uid')[feature].agg(agg))
            te[new_col] = te['uid'].map(tr.groupby('uid')[feature].agg(agg))

    tr = tr.drop(columns='uid')
    te = te.drop(columns='uid')

    return tr, te

train, test = create_features(train, test, ['DAYS_BIRTH', 'DAYS_EMPLOYED'], 'begin_month', ['mean', 'std', 'max'])
train, test = create_features(train, test, ['DAYS_BIRTH', 'DAYS_EMPLOYED'], 'begin_month_year', ['mean', 'std'])
train, test = create_features(train, test, ['DAYS_BIRTH', 'DAYS_EMPLOYED'], 'begin_month_month', ['mean', 'std'])
train, test = create_features(train, test, ['uid_rows'], 'begin_month_year', ['mean'])

# train, test = create_features(train, test, ['DAYS_BIRTH'], 'begin_month_month', ['mean'])
# train, test = create_features(train, test, ['DAYS_EMPLOYED'], 'DAYS_BIRTH', ['mean'])

# train, test = create_features(train, test, ['DAYS_BIRTH', 'DAYS_EMPLOYED_binary'], 'begin_month_month', ['mean'])
# train, test = create_features(train, test, ['uid_rows'], 'begin_month_month', ['mean',])
# train, test = create_features(train, test, 'DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED_binary', ['mean'])
# train, test = create_features(train, test, 'DAYS_BIRTH', 'DAYS_EMPLOYED_binary', 'begin_month_month', ['std'])
# train, test = create_features(train, test, 'DAYS_BIRTH', 'DAYS_EMPLOYED_DATE_month', 'begin_month_month', ['mean'])
# train, test = create_features(train, test, 'DAYS_BIRTH', 'occyp_type', 'begin_month', ['mean'])

missing_features = [col for col in train.columns if train[col].isnull().sum()>0]
print('missing_features :', missing_features)
for col in missing_features:
    train[col] = train[col].fillna(-99)
    test[col] = test[col].fillna(-99)

##### 현재 시점 2019년이라고 가정
drop_col = ['index', 'FLAG_MOBIL', 'DAYS_BIRTH_DATE', 'DAYS_EMPLOYED_DATE', 'begin_month_DATE', 'DAYS_EMPLOYED_binary', 'DAYS_EMPLOYED', 'DAYS_BIRTH',
            'begin_month_year', 'begin_month_month', 'uid_rows', 'rank']
            
